In [78]:
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

import os
import re
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [79]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

#initialize Lang Class
class Lang:
   def __init__(self):
       #initialize containers to hold the words and corresponding index
       self.word2index = {}
       self.word2count = {}
       self.index2word = {0: "SOS", 1: "EOS"}
       self.n_words = 2  # Count SOS and EOS

#split a sentence into words and add it to the container
   def addSentence(self, sentence):
       for word in sentence.split(' '):
           self.addWord(word)

#If the word is not in the container, the word will be added to it, 
#else, update the word counter
   def addWord(self, word):
       if word not in self.word2index:
           self.word2index[word] = self.n_words
           self.word2count[word] = 1
           self.index2word[self.n_words] = word
           self.n_words += 1
       else:
           self.word2count[word] += 1

In [80]:
#Normalize every sentence
def normalize_sentence(df, lang):
   sentence = df[lang].str.lower()
   sentence = sentence.str.replace('[^A-Za-z\s]+', '')
   sentence = sentence.str.normalize('NFD')
   sentence = sentence.str.encode('ascii', errors='ignore').str.decode('utf-8')
   return sentence

def read_sentence(df, lang1, lang2):
   sentence1 = normalize_sentence(df, lang1)
   sentence2 = normalize_sentence(df, lang2)
   return sentence1, sentence2

def read_file(loc, lang1, lang2):
   df = pd.read_csv(loc, delimiter='|', header=None, names=[lang1, lang2])
   return df

def process_data(lang1,lang2):
   df = read_file('training/%s-%s.txt' % (lang1, lang2), lang1, lang2)
   print("Read %s sentence pairs" % len(df))
   sentence1, sentence2 = read_sentence(df, lang1, lang2)

   source = Lang()
   target = Lang()
   pairs = []
   for i in range(len(df)):
       if len(sentence1[i].split(' ')) < MAX_LENGTH and len(sentence2[i].split(' ')) < MAX_LENGTH:
           full = [sentence1[i], sentence2[i]]
           source.addSentence(sentence1[i])
           target.addSentence(sentence2[i])
           pairs.append(full)

   return source, target, pairs

In [81]:
def indexesFromSentence(lang, sentence):
   return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
   indexes = indexesFromSentence(lang, sentence)
   indexes.append(EOS_token)
   return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(input_lang, output_lang, pair):
   input_tensor = tensorFromSentence(input_lang, pair[0])
   target_tensor = tensorFromSentence(output_lang, pair[1])
   return (input_tensor, target_tensor)

In [82]:
class Encoder(nn.Module):
   def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers):
       super(Encoder, self).__init__()
      
       #set the encoder input dimesion , embbed dimesion, hidden dimesion, and number of layers 
       self.input_dim = input_dim
       self.embbed_dim = embbed_dim
       self.hidden_dim = hidden_dim
       self.num_layers = num_layers

       #initialize the embedding layer with input and embbed dimention
       self.embedding = nn.Embedding(input_dim, self.embbed_dim)
       #intialize the GRU to take the input dimetion of embbed, and output dimention of hidden and
       #set the number of gru layers
       self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
              
   def forward(self, src):
      
       embedded = self.embedding(src).view(1,1,-1)
       outputs, hidden = self.gru(embedded)
       return outputs, hidden

class Decoder(nn.Module):
   def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers):
       super(Decoder, self).__init__()

#set the encoder output dimension, embed dimension, hidden dimension, and number of layers 
       self.embbed_dim = embbed_dim
       self.hidden_dim = hidden_dim
       self.output_dim = output_dim
       self.num_layers = num_layers

# initialize every layer with the appropriate dimension. For the decoder layer, it will consist of an embedding, GRU, a Linear layer and a Log softmax activation function.
       self.embedding = nn.Embedding(output_dim, self.embbed_dim)
       self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
       self.out = nn.Linear(self.hidden_dim, output_dim)
       self.softmax = nn.LogSoftmax(dim=1)
      
   def forward(self, input, hidden):

# reshape the input to (1, batch_size)
       input = input.view(1, -1)
       embedded = F.relu(self.embedding(input))
       output, hidden = self.gru(embedded, hidden)       
       prediction = self.softmax(self.out(output[0]))
      
       return prediction, hidden

class Seq2Seq(nn.Module):
   def __init__(self, encoder, decoder, device, MAX_LENGTH=MAX_LENGTH):
       super().__init__()
      
#initialize the encoder and decoder
       self.encoder = encoder
       self.decoder = decoder
       self.device = device
     
   def forward(self, source, target, teacher_forcing_ratio=0.5):

       input_length = source.size(0) #get the input length (number of words in sentence)
       batch_size = target.shape[1] 
       target_length = target.shape[0]
       vocab_size = self.decoder.output_dim
      
#initialize a variable to hold the predicted outputs
       outputs = torch.zeros(target_length, batch_size, vocab_size).to(self.device)

#encode every word in a sentence
       for i in range(input_length):
           encoder_output, encoder_hidden = self.encoder(source[i])

#use the encoder’s hidden layer as the decoder hidden
       decoder_hidden = encoder_hidden.to(device)
  
#add a token before the first predicted word
       decoder_input = torch.tensor([SOS_token], device=device)  # SOS

#topk is used to get the top K value over a list
#predict the output word from the current target word. If we enable the teaching force,  then the #next decoder input is the next word, else, use the decoder output highest value. 

       for t in range(target_length):   
           decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
           outputs[t] = decoder_output
           teacher_force = random.random() < teacher_forcing_ratio
           topv, topi = decoder_output.topk(1)
           input = (target[t] if teacher_force else topi)
           if(teacher_force == False and input.item() == EOS_token):
               break

       return outputs

In [83]:
teacher_forcing_ratio = 0.5

def clacModel(model, input_tensor, target_tensor, model_optimizer, criterion):
   model_optimizer.zero_grad()

   input_length = input_tensor.size(0)
   loss = 0
   epoch_loss = 0
   # print(input_tensor.shape)

   output = model(input_tensor, target_tensor)

   num_iter = output.size(0)
   print(num_iter)

#calculate the loss from a predicted sentence with the expected result
   for ot in range(num_iter):
       loss += criterion(output[ot], target_tensor[ot])

   loss.backward()
   model_optimizer.step()
   epoch_loss = loss.item() / num_iter

   return epoch_loss

def trainModel(model, source, target, pairs, num_iteration=20000):
   model.train()

   optimizer = optim.SGD(model.parameters(), lr=0.01)
   criterion = nn.NLLLoss()
   total_loss_iterations = 0

   training_pairs = [tensorsFromPair(source, target, random.choice(pairs))
                     for i in range(num_iteration)]
  
   for iter in range(1, num_iteration+1):
       training_pair = training_pairs[iter - 1]
       input_tensor = training_pair[0]
       target_tensor = training_pair[1]

       loss = clacModel(model, input_tensor, target_tensor, optimizer, criterion)

       total_loss_iterations += loss

       if iter % 5000 == 0:
           avarage_loss= total_loss_iterations / 5000
           total_loss_iterations = 0
           print('%d %.4f' % (iter, avarage_loss))
          
   torch.save(model.state_dict(), 'mytraining.pt')
   return model

In [ ]:
def evaluate(model, input_lang, output_lang, sentences, max_length=MAX_LENGTH):
   with torch.no_grad():
       input_tensor = tensorFromSentence(input_lang, sentences[0])
       output_tensor = tensorFromSentence(output_lang, sentences[1])
  
       decoded_words = []
  
       output = model(input_tensor, output_tensor)
       # print(output_tensor)
  
       for ot in range(output.size(0)):
           topv, topi = output[ot].topk(1)
           # print(topi)

           if topi[0].item() == EOS_token:
               decoded_words.append('<EOS>')
               break
           else:
               decoded_words.append(output_lang.index2word[topi[0].item()])
   return decoded_words

def evaluateRandomly(model, source, target, pairs, n=10):
   for i in range(n):
       pair = random.choice(pairs)
       print('source' + pair[0])
       #print(‘target {}’.format(pair[1]))
       output_words = evaluate(model, source, target, pair)
       output_sentence = ' '.join(output_words)
       #print(‘predicted {}’.format(output_sentence))

In [93]:
lang1 = 'demotic'
lang2 = 'english'
source, target, pairs = process_data(lang1, lang2)

randomize = random.choice(pairs)
print('random sentence {}'.format(randomize))

#print number of words
input_size = source.n_words
output_size = target.n_words
print('Input : {} Output : {}'.format(input_size, output_size))

embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 100000

#create encoder-decoder model
encoder = Encoder(input_size, hidden_size, embed_size, num_layers)
decoder = Decoder(output_size, hidden_size, embed_size, num_layers)

model = Seq2Seq(encoder, decoder, device).to(device)

#print model 
print(encoder)
print(decoder)

model = trainModel(model, source, target, pairs, num_iteration)
evaluateRandomly(model, source, target, pairs)

Read 800 sentence pairs
random sentence ['rir ptr p rb', 'to phatres the reading priest']
Input : 555 Output : 930


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


Encoder(
  (embedding): Embedding(555, 256)
  (gru): GRU(256, 512)
)
Decoder(
  (embedding): Embedding(930, 256)
  (gru): GRU(256, 512)
  (out): Linear(in_features=512, out_features=930, bias=True)
  (softmax): LogSoftmax(dim=1)
)
8
10
19
20
7
9
12
5
7
5
5
12
14
14
19
10
19
10
17
10
8
8
10
5
7
5
18
8
11
10
13
16
11
15
18
6
4
7
18
9
7
10
10
7
14
5
10
17
18
13
17
11
5
12
9
16
9
9
17
9
19
8
13
13
7
10
13
19
12
5
13
20
18
18
8
12
18
12
13
18
17
5
5
8
18
11
5
18
19
7
5
20
16
15
9
18
8
20
8
8
11
17
8
7
13
13
10
7
10
8
8
6
14
13
10
14
18
7
7
13
17
13
13
17
11
15
17
12
16
13
17
20
16
19
8
11
20
9
7
13
10
8
11
17
14
5
11
4
8
20
6
18
17
6
11
8
8
5
5
6
14
19
9
11
15
8
12
20
14
6
4
18
10
10
10
14
17
14
6
13
9
17
11
13
10
12
11
11
8
5
17
6
10
8
13
15
8
15
7
10
12
14
8
5
7
16
17
14
13
9
13
8
7
10
15
5
16
12
5
9
9
5
20
3
16
7
12
9
13
5
18
11
9
11
10
16
6
7
15
13
5
14
7
4
13
9
13
8
7
8
8
15
9
8
3
15
11
13
10
13
8
7
10
10
10
11
19
14
6
12
11
8
4
16
4
5
7
9
13
4
5
15
18
14
9
14
20
8
12
14
13
13
12
9
17


4
14
14
10
10
5
19
19
14
16
5
5
11
14
14
8
12
10
8
10
8
17
12
12
20
14
18
8
5
11
11
8
17
14
10
8
12
7
6
13
5
19
13
14
8
14
5
11
20
11
18
5
19
3
14
8
9
11
11
16
9
9
9
6
17
14
10
6
19
5
12
8
10
10
9
14
7
13
11
8
16
18
3
6
7
5
18
5
17
5
10
14
15
8
4
13
12
10
12
13
18
9
5
19
16
8
5
8
10
20
19
10
7
17
11
9
17
16
15
17
10
5
15
6
8
16
14
20
8
10
14
19
17
9
8
13
13
19
8
10
10
9
5
14
10
10
8
9
13
14
5
6
15
19
12
9
14
11
13
10
11
16
18
7
19
6
14
8
7
8
10
14
19
20
4
13
14
14
8
18
17
13
13
9
7
19
5
20
12
11
5
14
9
14
14
5
7
7
8
15
12
5
7
7
10
5
11
8
11
9
12
10
12
9
9
18
18
19
9
12
10
6
18
12
18
8
19
11
14
10
14
6
8
8
7
13
5
14
16
14
12
5
9
7
11
13
14
18
10
8
11
12
18
8
20
10
5
8
14
12
9
5
6
13
17
19
14
4
10
17
8
11
15
11
18
12
9
10
20
14
18
6
9
9
6
5
10
7
13
10
8
6
11
10
16
12
14
16
5
8
5
6
20
18
7
8
17
6
8
19
17
5
5
6
7
13
18
10
10
9
8
4
13
16
20
17
20
11
11
13
19
15
10
20
4
19
9
6
13
13
15
11
19
11
8
8
3
14
8
20
13
9
8
12
7
9
12
10
19
11
16
8
7
10
5
10
12
17
13
8
5
19
9
11
10
20
19
20
15
14
17
1

7
6
10
14
17
14
11
9
12
8
10
5
13
8
11
7
20
9
11
9
10
10
6
17
14
6
11
5
13
14
10
11
12
13
14
14
9
20
7
10
5
5
10
9
14
8
16
15
12
17
11
14
5
8
8
11
13
14
5
16
16
5
8
12
14
8
7
11
19
12
7
5
6
7
5
13
12
10
18
14
18
5
10
5
13
16
13
18
19
13
8
14
7
5
19
10
14
18
10
16
13
11
11
12
13
13
11
6
5
18
18
8
14
6
7
10
11
9
9
6
18
10
14
10
11
19
14
15
8
11
14
13
20
12
8
7
8
18
12
13
13
17
17
20
20
14
11
6
20
15
14
14
11
16
7
13
10
17
10
14
13
10
20
20
6
8
12
12
8
15
3
8
20
9
19
8
14
12
14
18
9
5
14
4
14
15
14
18
10
8
11
20
12
5
11
6
19
9
13
12
3
11
8
13
18
13
13
9
8
11
14
14
16
18
12
11
14
7
6
12
12
17
5
10
11
8
20
14
9
7
5
5
7
11
18
11
8
18
5
5
9
13
14
5
20
10
5
16
20
8
10
10
6
19
10
19
8
5
16
18
11
5
12
14
13
18
14
15
7
6
14
5
11
14
9
11
6
8
6
16
9
13
3
20
11
8
20
11
4
16
16
4
11
13
19
15
16
19
20
10
9
10
10
18
4
7
18
5
5
12
9
9
16
8
16
15
5
14
20
18
17
5
9
10
12
11
6
16
19
6
20
12
5
7
14
15
6
13
14
11
7
14
9
16
12
4
18
13
12
13
7
19
4
14
18
19
6
17
14
9
10
6
20
9
7
5
4
6
10
8
16
8
12
10
8
14
8
16

...

7
18
15
14
8
9
9
20
19
9
8
5
14
20
11
7
7
7
11
11
19
13
10
17
9
12
19
19
18
17
5
9
14
17
5
15
19
16
8
7
9
18
9
5
19
9
10
8
10
9
4
16
18
9
16
18
10
13
13
20
13
8
14
16
10
14
19
14
14
10
12
4
7
5
9
18
7
7
20
20
10
18
19
14
14
13
5
5
6
9
19
10
13
10
5
6
13
6
19
16
17
19
8
12
9
13
12
9
11
18
7
6
8
16
14
11
17
5
7
13
9
12
8
19
11
9
7
9
9
12
13
5
7
9
19
14
20
12
20
5
18
5
14
13
13
5
14
7
13
8
10
13
18
11
10
6
9
10
8
10
5
14
14
10
8
17
10
15
13
13
6
8
20
7
13
18
7
12
4
10
15
12
6
7
8
7
17
10
15
19
13
11
15
10
15
19
7
12
13
7
7
4
14
10
18
7
11
12
6
8
5
11
7
18
13
18
9
9
15
14
3
6
8
18
20
10
11
17
8
6
8
13
10
7
5
10
10
13
4
18
19
15
11
19
8
18
9
15
18
19
9
12
8
12
9
10
15
12
10
10
8
16
9
7
11
8
14
17
8
14
9
14
10
12
8
20
8
9
15
9
12
5
10
18
13
12
6
6
8
16
8
11
10
10
11
14
8
19
7
7
14
16
6
19
11
8
6
11
9
6
7
16
14
19
12
10
18
12
20
19
15
13
11
8
17
12
8
14
16
13
10
9
20
6
12
7
8
9
5
10
6
20
11
12
15
6
9
14
7
5
20
18
6
5
8
19
17
19
20
7
7
16
6
9
3
11
9
9
13
8
7
5
5
18
19
5
14
9
14
13
17
6
8
9
20


11
6
19
12
5
7
16
7
7
20
20
7
7
15
7
17
12
11
10
15
6
11
13
10
5
19
11
13
18
14
5
11
10
16
10
7
15
20
12
14
6
17
15
4
14
5
11
17
10
16
5
9
11
11
14
5
14
5
5
14
16
19
12
11
10
11
6
17
19
7
7
11
17
9
16
18
8
14
18
8
9
13
11
6
6
12
5
20
15
10
14
10
14
6
15
17
15
14
5
6
17
13
7
20
12
10
8
19
11
5
5
10
13
3
5
9
4
7
17
20
17
14
7
5
9
20
19
5
17
15
18
7
6
9
12
8
10
9
11
7
12
20
11
11
9
6
8
19
8
17
4
7
5
8
20
8
16
18
5
16
13
19
5
11
18
16
15
18
11
16
11
20
7
14
5
8
20
6
17
14
5
13
14
5
8
13
13
13
20
19
5
8
13
10
15
17
14
11
9
9
14
5
20
17
5
16
12
11
5
11
10
11
16
6
11
8
20
3
16
13
13
13
8
5
6
16
13
11
12
14
7
6
11
17
9
18
8
9
8
9
5
10
10
10
7
14
15
13
14
7
8
10
11
8
13
18
5
3
12
7
20
17
9
7
14
14
14
18
17
11
8
8
15
18
20
12
19
8
7
9
17
13
7
7
15
6
18
12
7
10
11
11
12
8
8
8
20
16
7
13
16
12
7
13
8
8
17
11
11
6
10
7
11
19
17
7
10
13
8
14
6
13
9
5
16
9
9
18
8
15
14
8
16
20
11
5
9
13
18
10
18
9
14
16
19
9
18
14
9
10
11
6
12
19
10
5
19
12
6
8
14
13
3
11
11
18
7
11
20
5
8
19
9
17
6
19
18
11
12
11
13

13
5
9
5
12
10
18
14
14
8
8
15
19
6
20
13
6
18
8
17
6
10
5
11
6
12
5
13
15
6
7
15
11
11
8
8
17
14
15
13
11
12
6
16
8
17
6
12
10
11
16
14
14
10
18
5
17
12
19
8
5
7
11
13
8
17
9
7
12
5
6
13
13
5
14
19
14
10
13
9
11
9
11
6
7
11
13
15
6
9
9
8
7
20
18
18
8
10
17
7
12
7
18
16
14
6
8
9
14
6
7
19
7
20
7
5
15
13
20
5
12
14
14
4
14
11
9
11
13
18
7
12
8
10
5
7
16
11
5
12
18
15
20
20
17
15
17
20
13
19
18
14
20
15
5
17
8
10
6
7
9
12
7
12
7
6
17
18
19
16
6
12
9
11
11
19
14
5
14
12
17
20
5
7
14
17
10
15
11
15
15
9
7
11
8
8
10
17
10
9
18
11
9
16
8
7
9
20
19
12
15
18
14
20
19
5
6
8
13
20
5
14
13
15
8
13
8
12
7
9
6
16
7
6
18
8
14
18
17
7
7
19
18
9
8
14
9
17
5
19
14
5
8
6
13
8
15
7
9
19
8
15
19
8
20
8
12
6
15
8
18
14
17
7
17
13
9
20
13
16
8
18
11
19
16
11
4
8
11
18
19
9
8
17
7
20
7
14
5
14
6
17
8
12
18
7
13
10
8
4
11
15
11
14
17
9
13
14
16
5
8
14
18
20
12
8
20
6
5
6
20
19
6
12
5
7
5
15
17
13
17
5
20
6
11
7
9
6
10
6
15
9
18
17
19
13
11
9
10
6
6
5
11
9
6
12
14
8
8
5
10
8
17
13
9
8
14
20
12
8
7
16
11
11
17


9
8
14
20
3
12
12
17
13
5
6
11
16
17
14
8
9
20
5
6
14
8
7
8
17
12
18
6
6
3
8
5
13
9
16
20
19
7
11
9
10
18
16
17
5
8
14
10
20
15
14
15
14
8
3
9
14
9
20
8
9
7
18
12
5
7
19
11
12
14
10
6
12
16
10
20
14
11
5
10
10
8
8
17
7
16
10
11
13
17
18
15
9
9
20
15
8
20
20
11
15
16
5
7
9
15
9
15
10
10
14
14
14
19
20
13
10
10
9
16
14
13
17
11
7
18
6
17
17
11
17
14
16
5
11
8
8
17
12
10
15
4
13
19
20
5
11
7
14
19
5
15
13
8
5
14
16
12
8
10
20
13
8
12
9
12
10
9
6
5
13
14
13
9
16
14
14
20
19
14
5
10
17
5
19
20
19
17
6
10
20
6
13
8
15
5
6
13
7
20
17
18
10
18
8
9
8
9
8
10
7
7
9
5
8
5
15
10
8
6
10
20
7
8
8
15
5
19
13
9
11
10
7
8
13
6
11
15
5
5
8
16
10
11
17
9
8
10
14
7
14
8
12
11
14
9
14
9
14
12
11
14
14
19
4
11
17
8
9
8
13
5
8
20
17
15
9
15
15
15
18
10
18
12
14
15
18
16
7
10
18
17
20
14
14
7
15
9
15
8
10
11
12
14
11
8
3
5
9
12
6
8
10
18
20
14
5
7
14
5
20
18
18
8
10
8
3
16
6
19
8
12
14
11
6
10
11
15
11
19
8
13
11
7
20
10
16
18
5
5
8
14
5
11
10
7
13
8
14
19
16
9
5
6
14
5
8
10
10
19
8
17
7
10
11
12
14
7
13
9
15


TypeError: evaluateRandomly() takes from 2 to 3 positional arguments but 4 were given